In [7]:
import os
local_finder_addition = "/Users/anna/sftp_mount/hpc_disk2/02-awegmann/"

In [59]:
path_gpt2_PAN = os.path.join(local_finder_addition, "TOKENIZER/output/VAR/train-mixed/base-BERT/mixed-gpt2-32000/749M/steps-45000/seed-42/42/PAN")
path_gpt2_AV = os.path.join(local_finder_addition, "TOKENIZER/output/VAR/train-mixed/base-BERT/mixed-gpt2-32000/749M/steps-45000/seed-42/42/sadiri")
path_no_AV = os.path.join(local_finder_addition, "TOKENIZER/output/VAR/train-mixed/base-BERT/mixed-no-32000/749M/steps-45000/seed-42/42/sadiri")
path_twitter_AV = os.path.join(local_finder_addition, "TOKENIZER/output/VAR/train-mixed/base-BERT/twitter-gpt2-32000/749M/steps-45000/seed-42/42/sadiri")
path_wikipedia_AV = os.path.join(local_finder_addition, "TOKENIZER/output/VAR/train-mixed/base-BERT/wikipedia-gpt2-32000/749M/steps-45000/seed-42/42/sadiri")
path_llama3_PAN = os.path.join(local_finder_addition, "TOKENIZER/output/VAR/train-mixed/base-BERT/mixed-llama3-32000/749M/steps-45000/seed-42/42/PAN")
path_wsorg_PAN = os.path.join(local_finder_addition, "TOKENIZER/output/VAR/train-mixed/base-BERT/mixed-wsorg-32000/749M/steps-45000/seed-42/42/PAN")
path_ws_PAN = os.path.join(local_finder_addition, "TOKENIZER/output/VAR/train-mixed/base-BERT/mixed-ws-32000/749M/steps-45000/seed-42/42/PAN")
path_128k_AV = os.path.join(local_finder_addition, "TOKENIZER/output/VAR/train-mixed/base-BERT/mixed-gpt2-128000/749M/steps-45000/seed-42/42/sadiri")
tokenizer_gpt2 = os.path.join(local_finder_addition, "TOKENIZER/tokenizer/mixed-gpt2-32000")
tokenizer_llama3 = os.path.join(local_finder_addition, "TOKENIZER/tokenizer/mixed-llama3-32000")
tokenizer_wsorg = os.path.join(local_finder_addition, "TOKENIZER/tokenizer/mixed-wsorg-32000")
tokenizer_ws = os.path.join(local_finder_addition, "TOKENIZER/tokenizer/mixed-ws-32000")
tokenizer_no = os.path.join(local_finder_addition, "TOKENIZER/tokenizer/mixed-no-32000")
tokenizer_500 = os.path.join(local_finder_addition, "TOKENIZER/tokenizer/mixed-gpt2-500")
tokenizer_4k = os.path.join(local_finder_addition, "TOKENIZER/tokenizer/mixed-gpt2-4000")
tokenizer_64k = os.path.join(local_finder_addition, "TOKENIZER/tokenizer/mixed-gpt2-64000")
tokenier_128k = os.path.join(local_finder_addition, "TOKENIZER/tokenizer/mixed-gpt2-128000")
tokenizer_twitter = os.path.join(local_finder_addition, "TOKENIZER/tokenizer/twitter-gpt2-32000")
tokenizer_pubmed = os.path.join(local_finder_addition, "TOKENIZER/tokenizer/pubmed-gpt2-32000")
tokenizer_wikipedia = os.path.join(local_finder_addition, "TOKENIZER/tokenizer/wikipedia-gpt2-32000")
path_gpt2_webbook_PAN = os.path.join(local_finder_addition, "TOKENIZER/output/VAR/base-BERT/mixed-gpt2-32000/749M/steps-45000/seed-42/42/PAN")
path_llama3_webbook_PAN = os.path.join(local_finder_addition, "TOKENIZER/output/VAR/base-BERT/mixed-llama3-32000/749M/steps-45000/seed-42/42/PAN")

In [71]:
path_wiki_NUCLE = os.path.join(local_finder_addition, "TOKENIZER/output/VAR/train-mixed/base-BERT/wikipedia-gpt2-32000/749M/steps-45000/seed-42/42/NUCLE")
path_mixed_NUCLE = os.path.join(local_finder_addition, "TOKENIZER/output/VAR/train-mixed/base-BERT/mixed-gpt2-32000/749M/steps-45000/seed-42/42/NUCLE")

In [23]:
# laod tokeinzer
from train_bert import load_tokenizer
no_tok, _ = load_tokenizer(tokenizer_no)
wsorg_tok, _ = load_tokenizer(tokenizer_wsorg)
ws_tok, _ = load_tokenizer(tokenizer_ws)
gpt2_tok, _ = load_tokenizer(tokenizer_gpt2)
llama3_tok, _ = load_tokenizer(tokenizer_llama3)
tok_500, _ = load_tokenizer(tokenizer_500)
tok_4k, _ = load_tokenizer(tokenizer_4k)
tok_64k, _ = load_tokenizer(tokenizer_64k)
tok_128k, _ = load_tokenizer(tokenier_128k)
tok_twitter, _ = load_tokenizer(tokenizer_twitter)
tok_pubmed, _ = load_tokenizer(tokenizer_pubmed)
tok_wikipedia, _ = load_tokenizer(tokenizer_wikipedia)

TOKENIZERS = {"gpt2": gpt2_tok, "llama3": llama3_tok, "wsorg": wsorg_tok, "ws": ws_tok, "no": no_tok, "500": tok_500, "4k": tok_4k, "64k": tok_64k, "128k": tok_128k, "twitter": tok_twitter, "pubmed": tok_pubmed, "wikipedia": tok_wikipedia}

2025-02-09 14:19:55,793 - styletokenizer.utility.custom_logger - DEBUG - Loading previously fitted tokenizer from local path: /Users/anna/sftp_mount/hpc_disk2/02-awegmann/TOKENIZER/tokenizer/mixed-no-32000/tokenizer.json
2025-02-09 14:19:55,852 - styletokenizer.utility.custom_logger - DEBUG - Loading previously fitted tokenizer from local path: /Users/anna/sftp_mount/hpc_disk2/02-awegmann/TOKENIZER/tokenizer/mixed-wsorg-32000/tokenizer.json
2025-02-09 14:19:55,910 - styletokenizer.utility.custom_logger - DEBUG - Loading previously fitted tokenizer from local path: /Users/anna/sftp_mount/hpc_disk2/02-awegmann/TOKENIZER/tokenizer/mixed-ws-32000/tokenizer.json
2025-02-09 14:19:55,970 - styletokenizer.utility.custom_logger - DEBUG - Loading previously fitted tokenizer from local path: /Users/anna/sftp_mount/hpc_disk2/02-awegmann/TOKENIZER/tokenizer/mixed-gpt2-32000/tokenizer.json
2025-02-09 14:19:56,032 - styletokenizer.utility.custom_logger - DEBUG - Loading previously fitted tokenizer fr

In [286]:
for key, tok in TOKENIZERS.items():
    print(f'Tokenized {key}:')
    print(tok.tokenize(" thiis teh lgbtq jumper")) # St. ma'am /home  SAY CAN Hello 1) ballots: narcotics Guy c00l lymphadenopathy')}')
    # ain't noboday got time for that.
    # lgbtq+ y'all queer b4 ;\n 5 a.m.  lgbtq+ authorised  559  tumour
    # and/or well-being 
    # endl;\n
    #  X-ray multilingual ballots in Europe/the US
    # partisan
    # in the U.S. Mr. e.g. en.wikipedia.org/wiki U.S.   monk /home
    # e-mail  
    #  
    # make-up
    # 588
    # Difference gpt2/llama3: 
    #   Minorities: queer, 
    #   British vs American: jelly, tumour
    # Difference 32k and 128k: 
    #   Minorities: lgbtq+
    #   Dialectal differences: jumper, trainers, yall, biscuit 
    #   youth lang: lmao, bruh, noo
    #   content words: ballots
    # Difference Twitter vs. Wikipedia
    #   Minorities: lgbtq+
    #   Dialectal differences: yall, aint, cant
    #   Youth lang: lmao, lol, bruh, noo, OMG
    #   content words: ballots, precursor
    # Difference ws merge character categories
    #   internet/youth language: ok. b4 y'all

Tokenized gpt2:
['Ġth', 'i', 'is', 'Ġte', 'h', 'Ġl', 'g', 'bt', 'q', 'Ġj', 'umper']
Tokenized llama3:
['Ġth', 'i', 'is', 'Ġte', 'h', 'Ġl', 'g', 'bt', 'q', 'Ġj', 'umper']
Tokenized wsorg:
['Ġ', 'th', 'i', 'is', 'Ġ', 'te', 'h', 'Ġ', 'lg', 'bt', 'q', 'Ġ', 'jum', 'per']
Tokenized ws:
['Ġth', 'i', 'is', 'Ġte', 'h', 'Ġl', 'g', 'bt', 'q', 'Ġj', 'umper']
Tokenized no:
['Ġth', 'i', 'is', 'Ġte', 'hĠ', 'l', 'g', 'bt', 'qĠ', 'j', 'um', 'per']
Tokenized 500:
['Ġth', 'i', 'is', 'Ġt', 'e', 'h', 'Ġl', 'g', 'b', 't', 'q', 'Ġj', 'um', 'p', 'er']
Tokenized 4k:
['Ġth', 'i', 'is', 'Ġte', 'h', 'Ġl', 'g', 'bt', 'q', 'Ġj', 'um', 'per']
Tokenized 64k:
['Ġthi', 'is', 'Ġteh', 'Ġlg', 'bt', 'q', 'Ġjumper']
Tokenized 128k:
['Ġthi', 'is', 'Ġteh', 'Ġlgbt', 'q', 'Ġjumper']
Tokenized twitter:
['Ġthi', 'is', 'Ġte', 'h', 'Ġlgbt', 'q', 'Ġj', 'umper']
Tokenized pubmed:
['Ġthi', 'is', 'Ġte', 'h', 'Ġl', 'g', 'b', 't', 'q', 'Ġj', 'um', 'per']
Tokenized wikipedia:
['Ġth', 'i', 'is', 'Ġte', 'h', 'Ġl', 'g', 'b', 't', 'q', 'Ġj', 

In [109]:
# loading classifications for tasks requiring sensitivity to language variation
import pandas as pd
df_gpt2 = pd.read_csv(os.path.join(path_gpt2_PAN, "2025-01-21_eval_dataset.tsv"), sep="\t")
df_llama3 = pd.read_csv(os.path.join(path_llama3_PAN, "2025-01-22_eval_dataset.tsv"), sep="\t")
df_wsorg = pd.read_csv(os.path.join(path_wsorg_PAN, "2025-01-25_eval_dataset.tsv"), sep="\t")
df_ws = pd.read_csv(os.path.join(path_ws_PAN, "2025-01-26_eval_dataset.tsv"), sep="\t")

df_no_av = pd.read_csv(os.path.join(path_no_AV, "eval_dataset.tsv"), sep="\t")

df_wb_gpt2 = pd.read_csv(os.path.join(path_gpt2_webbook_PAN, "2025-01-10_eval_dataset.tsv"), sep="\t")
df_wb_llama3 = pd.read_csv(os.path.join(path_llama3_webbook_PAN, "2025-01-10_eval_dataset.tsv"), sep="\t")

In [62]:
df_gpt2_av = pd.read_csv(os.path.join(path_gpt2_AV, "eval_dataset.tsv"), sep="\t")

In [33]:
df_no_av = pd.read_csv(os.path.join(path_no_AV, "eval_dataset.tsv"), sep="\t")

In [27]:
df_twitter_av = pd.read_csv(os.path.join(path_twitter_AV, "eval_dataset.tsv"), sep="\t")
df_wikipedia_av = pd.read_csv(os.path.join(path_wikipedia_AV, "eval_dataset.tsv"), sep="\t")

In [60]:
df_128k = pd.read_csv(os.path.join(path_128k_AV, "eval_dataset.tsv"), sep="\t")

In [72]:
df_wiki_NUCLE = pd.read_csv(os.path.join(path_wiki_NUCLE, "2025-01-23_eval_dataset.tsv"), sep="\t")
df_mixed_NUCLE = pd.read_csv(os.path.join(path_mixed_NUCLE, "2025-01-23_eval_dataset.tsv"), sep="\t")

In [64]:
# find cases where llama3 predicted correctly, but gpt2 did not
df_llama3_correct_gpt2_wrong = df_llama3[(df_llama3["label"] == df_llama3["predictions"]) & (df_gpt2["label"] != df_gpt2["predictions"])]
df_wb_gpt2_correct_llama3_wrong = df_wb_gpt2[(df_wb_gpt2["label"] == df_wb_gpt2["predictions"]) & (df_llama3["label"] != df_llama3["predictions"])]
df_ws_correct_gpt2_wrong = df_ws[(df_ws["label"] == df_ws["predictions"]) & (df_gpt2["label"] != df_gpt2["predictions"])]

In [11]:
df_no_correct_gpt2_wrong_AV = df_no_av[(df_no_av["label"] == df_no_av["predictions"]) & (df_gpt2_av["label"] != df_gpt2_av["predictions"])]
print(len(df_no_correct_gpt2_wrong_AV))

745


In [52]:
def compare_predictions(df_correct, df_incorrect, first_tok, second_tok, first_name, second_name, text1="text 1", text2="text 2", classification_at_1="Same Author", n_examples=10, nbr_char_to_show=300):
    i = 0
    # parse label and predictions
    df_first_correct_second_wrong = df_correct[(df_correct["label"] == df_correct["predictions"]) & (df_incorrect["label"] != df_incorrect["predictions"])]
    for index, row in df_first_correct_second_wrong.iterrows():
        print(f"=={classification_at_1}==" if row["predictions"] == 1 else f"==not {classification_at_1}==")
        print(f"Text 1: {row[text1][:nbr_char_to_show]}")
        print(f"\t Tokenized {first_name}:{' '.join(first_tok.tokenize(row[text1][:nbr_char_to_show]))}")
        print(f"\t Tokenized {second_name}:{' '.join(second_tok.tokenize(row[text1][:nbr_char_to_show]))}")
        print(f"\t Unique Tokens {first_name}: {set(first_tok.tokenize(row[text1])) - set(second_tok.tokenize(row[text1]))}")
        print(f"\t Unique Tokens {second_name}: {set(second_tok.tokenize(row[text1])) - set(first_tok.tokenize(row[text1]))}")
        if text2 is not None:
            print(f"Text 2: {row[text2][:nbr_char_to_show]}")   
            print(f"\t Tokenized {first_name}:{' '.join(first_tok.tokenize(row[text2][:nbr_char_to_show]))}")
            print(f"\t Tokenized {second_name}:{' '.join(second_tok.tokenize(row[text2][:nbr_char_to_show]))}")
            print(f"\t Unique Tokens {first_name}: {set(first_tok.tokenize(row[text2])) - set(second_tok.tokenize(row[text2]))}")
            print(f"\t Unique Tokens {second_name}: {set(second_tok.tokenize(row[text2])) - set(first_tok.tokenize(row[text2]))}")
        i += 1

        if i > n_examples: break

In [34]:
compare_predictions(df_twitter_av, df_wikipedia_av, tok_twitter, tok_wikipedia, "twitter", "wiki", classification_at_1="Same Author", text1="query_text", text2="candidate_text", n_examples=10)

==Same Author==
Text 1: size 4. This size is generally available in sizes of 12mm (½ in), 16mm (  in), 20mm (¾ in), 25mm (1 in), 30mm (1¼ in) and 40mm (1½ in).
 A 3.5mm in metric is equivalent to an imperial gauge size 6. This size is generally available in sizes of 12mm (½ in), 16mm (5/3 in), 20mm (¾ in), 25mm (1 in), 30m
	 Tokenized twitter:s ize Ġ4 . ĠThis Ġsize Ġis Ġgenerally Ġavailable Ġin Ġs izes Ġof Ġ12 mm Ġ( Â ½ Ġin ), Ġ16 mm Ġ( Ġ Ġin ), Ġ20 mm Ġ( Â ¾ Ġin ), Ġ25 mm Ġ( 1 Ġin ), Ġ30 mm Ġ( 1 Â ¼ Ġin ) Ġand Ġ40 mm Ġ( 1 Â ½ Ġin ). Ċ ĠA Ġ3 . 5 mm Ġin Ġmet ric Ġis Ġequivalent Ġto Ġan Ġim perial Ġg aug e Ġsize Ġ6 . ĠThis Ġsize Ġis Ġgenerally Ġavailable Ġin Ġs izes Ġof Ġ12 mm Ġ( Â ½ Ġin ), Ġ16 mm Ġ( 5 / 3 Ġin ), Ġ20 mm Ġ( Â ¾ Ġin ), Ġ25 mm Ġ( 1 Ġin ), Ġ30 m
	 Tokenized wiki:size Ġ4 . ĠThis Ġsize Ġis Ġgenerally Ġavailable Ġin Ġsizes Ġof Ġ12 mm Ġ( Â½ Ġin ), Ġ16 mm Ġ( Ġ Ġin ), Ġ20 mm Ġ( Â ¾ Ġin ), Ġ25 mm Ġ( 1 Ġin ), Ġ30 mm Ġ( 1 Â ¼ Ġin ) Ġand Ġ40 mm Ġ( 1 Â½ Ġin ). Ċ ĠA Ġ3 . 5 mm Ġin Ġmetri

In [37]:
compare_predictions(df_no_av, df_wikipedia_av, no_tok, tok_wikipedia, "no", "wiki", classification_at_1="Same Author", text1="query_text", text2="candidate_text", n_examples=10)

==Same Author==
Text 1: . <PERSON> to <PERSON>, 26 June 1927, Phelps Stokes Papers, Yale.
 . <PERSON>, "Impressions of the Port Elizabeth Conference," _Universitas_ 8:8 (August 1928).
 . <PERSON> to <PERSON>, 24 October 1928, NAACP Papers, Library of Congress.
 ### NOTES TO CHAPTER 4
 . "<PERSON> of Africa Is Guest of the 
	 Tokenized no:.Ġ < P ERS ON > ĠtoĠ < P ERS ON >,Ġ 26Ġ JuneĠ 19 27 ,ĠP helpsĠ St ok esĠ Pap ers,Ġ Y al e.Ċ Ġ .Ġ < P ERS ON > ,Ġ" Im pression sĠofĠtheĠ PortĠ ElizabethĠ Con fer ence ,"Ġ _ Universit as _Ġ 8: 8 Ġ( AugustĠ 19 28 ).Ċ Ġ .Ġ < P ERS ON > ĠtoĠ < P ERS ON >,Ġ 24Ġ OctoberĠ 19 28 ,Ġ NA AC PĠ Pap ers,Ġ Libr aryĠofĠ Congres s.Ċ Ġ ###Ġ NOTE SĠ TOĠ CHAPTERĠ 4 ĊĠ .Ġ" < P ERS ON >Ġ ofĠ Afric aĠ IsĠ Gu estĠ ofĠtheĠ
	 Tokenized wiki:. Ġ< P ERS ON > Ġto Ġ< P ERS ON > , Ġ26 ĠJune Ġ1927 , ĠPhel ps ĠStokes ĠPapers , ĠYale . Ċ Ġ. Ġ< P ERS ON > , Ġ" Imp ress ions Ġof Ġthe ĠPort ĠElizabeth ĠConference ," Ġ _ Univers itas _ Ġ8 : 8 Ġ( August Ġ1928 ). Ċ Ġ. Ġ< P ERS ON > Ġto Ġ< P 

In [53]:
compare_predictions(df_llama3, df_gpt2, llama3_tok, gpt2_tok, "llama3", "gpt2", classification_at_1="Different Author", text1="text 1", text2="text 2", n_examples=10, nbr_char_to_show=-1)

==not Different Author==
Text 1: Agreed. I guess Brazil still trips me up though because there’s no real reason for them not to take a stronger stance either. One Brazilian told me they couldn’t “risk trade being impacted with Russia”, but I don’t buy that given that 81% of Brazil’s trade is done with 25 nations, and Russia isn’t one of those. At best Russian goods constitute less than .8% of Brazil’s trade. And for countries that constantly deride colonialism and imperialism and the evils of it and how all of their problems are because of colonial invasions and imperialist meddling, it just strikes me that a literal imperialist nation invading a small, weaker one purely as a land grab is casted as “not our problem” or “a European issue”. Like how would any of these countries feel if they got invaded by the US or China and Europe/the US/the rest of the world said “that’s a Latin American problem”
	 Tokenized llama3:Ag reed . ĠI Ġguess ĠBrazil Ġstill Ġtrips Ġme Ġup Ġthough Ġbecause Ġthe

In [64]:
compare_predictions(df_128k, df_gpt2_av, tok_128k, gpt2_tok, "128k", "32k", classification_at_1="Same Author", text1="query_text", text2="candidate_text", n_examples=10)

==not Same Author==
Text 1: close enough to see the coast. He didn't tell her that he'd quit the salt mines because, sea air notwithstanding, he could not bear to be so far underground, the white walls dwarfing him, his own shadow a stranger as he hunched over and shuttled carts of supplies to white men similar to how he imagi
	 Tokenized 128k:close Ġenough Ġto Ġsee Ġthe Ġcoast . ĠHe Ġdidn 't Ġtell Ġher Ġthat Ġhe 'd Ġquit Ġthe Ġsalt Ġmines Ġbecause , Ġsea Ġair Ġnotwithstanding , Ġhe Ġcould Ġnot Ġbear Ġto Ġbe Ġso Ġfar Ġunderground , Ġthe Ġwhite Ġwalls Ġdwarf ing Ġhim , Ġhis Ġown Ġshadow Ġa Ġstranger Ġas Ġhe Ġhunched Ġover Ġand Ġshutt led Ġcarts Ġof Ġsupplies Ġto Ġwhite Ġmen Ġsimilar Ġto Ġhow Ġhe Ġimag i
	 Tokenized 32k:close Ġenough Ġto Ġsee Ġthe Ġcoast . ĠHe Ġdidn 't Ġtell Ġher Ġthat Ġhe 'd Ġquit Ġthe Ġsalt Ġmines Ġbecause , Ġsea Ġair Ġnot with standing , Ġhe Ġcould Ġnot Ġbear Ġto Ġbe Ġso Ġfar Ġunderground , Ġthe Ġwhite Ġwalls Ġdwarf ing Ġhim , Ġhis Ġown Ġshadow Ġa Ġstranger Ġas Ġhe Ġh

In [ ]:
compare_predictions(df_wiki_NUCLE, df_mixed_NUCLE, tok_wikipedia, gpt2_tok, "wiki", "mixed", classification_at_1="Same Author", text1="query_text", text2="candidate_text", n_examples=10)

In [14]:
# note that 0 for PAN corresponds to same author or no change, see https://pan.webis.de/clef24/pan24-web/style-change-detection.html
i = 0
first_tok = gpt2_tok
first_name = "GPT2"
second_tok = no_tok
second_name = "no"
text1 = "query_text"
text2 = "candidate_text"
for index, row in df_no_correct_gpt2_wrong_AV.iterrows():
    print("==Same Author==" if row["predictions"] == 1 else "==Different Author==")
    print(f"Text 1: {row[text1][:300]}")
    print(f"\t Tokenized {first_name}:{' '.join(first_tok.tokenize(row[text1][:300]))}")
    print(f"\t Tokenized {second_name}:{' '.join(second_tok.tokenize(row[text1][:300]))}")
    print(f"\t Unique Tokens {first_name}: {set(first_tok.tokenize(row[text1])) - set(second_tok.tokenize(row[text1]))}")
    print(f"\t Unique Tokens {second_name}: {set(second_tok.tokenize(row[text1])) - set(first_tok.tokenize(row[text1]))}")
    print(f"Text 2: {row[text2][:300]}")   
    print(f"\t Tokenized {first_name}:{' '.join(first_tok.tokenize(row[text2][:300]))}")
    print(f"\t Tokenized {second_name}:{' '.join(second_tok.tokenize(row[text2][:300]))}")
    print(f"\t Unique Tokens {first_name}: {set(first_tok.tokenize(row[text2])) - set(second_tok.tokenize(row[text2]))}")
    print(f"\t Unique Tokens {second_name}: {set(second_tok.tokenize(row[text2])) - set(first_tok.tokenize(row[text2]))}")
    i += 1
    
    if i > 10: break

==Same Author==
Text 1: . <PERSON> to <PERSON>, 26 June 1927, Phelps Stokes Papers, Yale.
 . <PERSON>, "Impressions of the Port Elizabeth Conference," _Universitas_ 8:8 (August 1928).
 . <PERSON> to <PERSON>, 24 October 1928, NAACP Papers, Library of Congress.
 ### NOTES TO CHAPTER 4
 . "<PERSON> of Africa Is Guest of the 
	 Tokenized GPT2:. Ġ< PER SON > Ġto Ġ< PER SON >, Ġ26 ĠJune Ġ19 27 , ĠP hel ps ĠSto kes ĠP apers , ĠY ale . Ċ Ġ. Ġ< PER SON >, Ġ" Im press ions Ġof Ġthe ĠPort ĠElizabeth ĠConference ," Ġ_ Univers itas _ Ġ8 : 8 Ġ( August Ġ19 28 ). Ċ Ġ. Ġ< PER SON > Ġto Ġ< PER SON >, Ġ24 ĠOctober Ġ19 28 , ĠNA AC P ĠP apers , ĠLibrary Ġof ĠCongress . Ċ Ġ# ## ĠNOT ES ĠTO ĠCHAPTER Ġ4 Ċ Ġ. Ġ"< PER SON > Ġof ĠAfrica ĠIs ĠGu est Ġof Ġthe Ġ
	 Tokenized no:.Ġ < P ERS ON > ĠtoĠ < P ERS ON >,Ġ 26Ġ JuneĠ 19 27 ,ĠP helpsĠ St ok esĠ Pap ers,Ġ Y al e.Ċ Ġ .Ġ < P ERS ON > ,Ġ" Im pression sĠofĠtheĠ PortĠ ElizabethĠ Con fer ence ,"Ġ _ Universit as _Ġ 8: 8 Ġ( AugustĠ 19 28 ).Ċ Ġ .Ġ < P ERS ON > ĠtoĠ < P 

In [65]:
print(len(df_llama3_correct_gpt2_wrong))
print(len(df_wb_gpt2_correct_llama3_wrong))
print(len(df_ws_correct_gpt2_wrong))

531
434
447


In [49]:
# note that 0 for PAN corresponds to same author or no change, see https://pan.webis.de/clef24/pan24-web/style-change-detection.html
i = 0
first_tok = gpt2_tok
first_name = "GPT2"
second_tok = llama3_tok
second_name = "LLAMA3"
# second_tok = ws_tok
# second_name = "ws"
for index, row in df_ws_correct_gpt2_wrong.iterrows():
    print("==Same Author==" if row["predictions"] == 0 else "==Different Author==")
    print(f"Text 1: {row['text 1'][:300]}")
    print(f"\t Tokenized {first_name}:{' '.join(first_tok.tokenize(row['text 1'][:300]))}")
    print(f"\t Tokenized {second_name}:{' '.join(second_tok.tokenize(row['text 1'][:300]))}")
    print(f"\t Unique Tokens {first_name}: {set(first_tok.tokenize(row['text 1'])) - set(second_tok.tokenize(row['text 1']))}")
    print(f"\t Unique Tokens {second_name}: {set(second_tok.tokenize(row['text 1'])) - set(first_tok.tokenize(row['text 1']))}")
    print(f"Text 2: {row['text 2'][:300]}")   
    print(f"\t Tokenized {first_name}:{' '.join(first_tok.tokenize(row['text 2'][:300]))}")
    print(f"\t Tokenized {second_name}:{' '.join(second_tok.tokenize(row['text 2'][:300]))}")
    print(f"\t Unique Tokens {first_name}: {set(first_tok.tokenize(row['text 2'])) - set(second_tok.tokenize(row['text 2']))}")
    print(f"\t Unique Tokens {second_name}: {set(second_tok.tokenize(row['text 2'])) - set(first_tok.tokenize(row['text 2']))}")
    i += 1
    
    if i > 10: break

NameError: name 'df_ws_correct_gpt2_wrong' is not defined

In [67]:
# load webbook corpus
path_webbook = os.path.join(local_finder_addition,
                             "TOKENIZER/data/train-corpora/webbook")

In [68]:
from datasets import load_from_disk
webbook = load_from_disk(path_webbook)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Loading dataset from disk:   0%|          | 0/40 [00:00<?, ?it/s]

In [70]:
# for the dev set print the first 10 texts
for i in range(10):
    print(f"Excerpt {i}: {webbook['train'][i]['text'][:300]}")

Excerpt 0:  visit and they all swore on a second blood oath that it wasn't them.

"What about Phantom?" Hell Girl asked. "Maybe Phantom talked to Campus News."

Frankenstein shook his head. "Why? Skeletor has pictures of Phantom like he has of the rest of us."

Ghost Face looked thoughtful. "Maybe Phantom want
Excerpt 1: The future of SA-affiliated club sports, a cappella, and Greek groups is uncertain after the All-Campus Judicial Council ruled Friday that they may not use gendered language in their constitutions, advertisements, and names, and that they may not participate in gender-exclusive competitions.

The ru
Excerpt 2:  to support it; then a chunk of Iceland spar was discovered among navigational instruments on an Elizabethan ship that sank in 1592. This wasn't a Viking ship, but it existed almost two hundred years before scientists even understood the concept of using a crystal to determine direction at dawn and 
Excerpt 3:  in his Trojan maiden. Statius's Antigone, like all t

In [71]:
path_gpt2_mnli = os.path.join(local_finder_addition, "TOKENIZER/output/GLUE/train-mixed/base-BERT/mixed-gpt2-32000/749M/steps-45000/seed-42/42/mnli")
path_llama3_mnli = os.path.join(local_finder_addition, "TOKENIZER/output/GLUE/train-mixed/base-BERT/mixed-llama3-32000/749M/steps-45000/seed-42/42/mnli")
df_mnli_gpt2 = pd.read_csv(os.path.join(path_gpt2_mnli, "2025-02-01_eval_dataset_mnli.tsv"), sep="\t")
df_mnli_llama3 = pd.read_csv(os.path.join(path_llama3_mnli, "2025-01-22_eval_dataset_mnli.tsv"), sep="\t")

In [72]:
df_mnli_gpt2_correct_llama3_wrong = df_mnli_gpt2[(df_mnli_gpt2["label"] == df_mnli_gpt2["predictions"]) & (df_mnli_llama3["label"] != df_mnli_llama3["predictions"])]

In [73]:
print(len(df_mnli_gpt2_correct_llama3_wrong))

619


In [79]:
i = 0
first_tok = gpt2_tok
first_name = "GPT2"
second_tok = llama3_tok
second_name = "LLAMA3"
for index, row in df_mnli_gpt2_correct_llama3_wrong.iterrows():
    if row["predictions"] == 0:
        print("==entailment==")
    elif row["predictions"] == 1:
        print("==neutral==")
    elif row["predictions"] == 2:
        print("==contradiction==")
    print(f"Text 1: {row['premise'][:300]}")
    print(f"\t Tokenized {first_name}:{' '.join(first_tok.tokenize(row['premise'][:300]))}")
    print(f"\t Tokenized {second_name}:{' '.join(second_tok.tokenize(row['premise'][:300]))}")
    print(f"\t Unique Tokens {first_name}: {set(first_tok.tokenize(row['premise'])) - set(second_tok.tokenize(row['premise']))}")
    print(f"\t Unique Tokens {second_name}: {set(second_tok.tokenize(row['premise'])) - set(first_tok.tokenize(row['premise']))}")
    print(f"Text 2: {row['hypothesis'][:300]}")   
    print(f"\t Tokenized {first_name}:{' '.join(first_tok.tokenize(row['hypothesis'][:300]))}")
    print(f"\t Tokenized {second_name}:{' '.join(second_tok.tokenize(row['hypothesis'][:300]))}")
    print(f"\t Unique Tokens {first_name}: {set(first_tok.tokenize(row['hypothesis'])) - set(second_tok.tokenize(row['hypothesis']))}")
    print(f"\t Unique Tokens {second_name}: {set(second_tok.tokenize(row['hypothesis'])) - set(first_tok.tokenize(row['hypothesis']))}")
    i += 1
    
    if 20 > i > 50: break

==entailment==
Text 1: uh i don't know i i have mixed emotions about him uh sometimes i like him but at the same times i love to see somebody beat him
	 Tokenized GPT2:uh Ġi Ġdon 't Ġknow Ġi Ġi Ġhave Ġmixed Ġemotions Ġabout Ġhim Ġuh Ġsometimes Ġi Ġlike Ġhim Ġbut Ġat Ġthe Ġsame Ġtimes Ġi Ġlove Ġto Ġsee Ġsomebody Ġbeat Ġhim
	 Tokenized LLAMA3:uh Ġi Ġdon 't Ġknow Ġi Ġi Ġhave Ġmixed Ġemotions Ġabout Ġhim Ġuh Ġsometimes Ġi Ġlike Ġhim Ġbut Ġat Ġthe Ġsame Ġtimes Ġi Ġlove Ġto Ġsee Ġsomebody Ġbeat Ġhim
	 Unique Tokens GPT2: set()
	 Unique Tokens LLAMA3: set()
Text 2: I like him for the most part, but would still enjoy seeing someone beat him.
	 Tokenized GPT2:I Ġlike Ġhim Ġfor Ġthe Ġmost Ġpart , Ġbut Ġwould Ġstill Ġenjoy Ġseeing Ġsomeone Ġbeat Ġhim .
	 Tokenized LLAMA3:I Ġlike Ġhim Ġfor Ġthe Ġmost Ġpart , Ġbut Ġwould Ġstill Ġenjoy Ġseeing Ġsomeone Ġbeat Ġhim .
	 Unique Tokens GPT2: set()
	 Unique Tokens LLAMA3: set()
==contradiction==
Text 1: yeah i i think my favorite restaurant is always been